In [1]:
# IMPORTING LIBRARIES
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [80]:
# FUNCTIONS
def get_title(soup):
    try:
        title = soup.find('span', attrs={'id':'productTitle'}).text.strip()
    except AttributeError:
        title = ""
    return title

def get_price(soup):
    try:
        price = soup.find('span', attrs={'class':'a-price a-text-price a-size-medium apexPriceToPay'}).span.text
    except AttributeError:            
        price = ""
    return price

def get_rating(soup):
    try:
        rating = soup.find('span', attrs={'class':'a-size-base a-color-base'}).text
    except AttributeError:
        rating = ""
    return rating


def get_review_count(soup):
    try:
        review_count = soup.find('span', attrs={'id':'acrCustomerReviewText'}).text
    except AttributeError:
        review_count = ""
    return review_count

def get_availability(soup):
    try:
        available = soup.find('span', attrs={'class':'a-size-base a-color-price a-text-bold'}).text
    except AttributeError:
        available = "Not Available"
    return available


    

In [96]:
# MIAN FUNCTION
if __name__ == '__main__':
    # CREDENTIALS
    URL = "https://www.amazon.com/s?k=playstation+5&crid=Q4PL43VKZ4MP&sprefix=playst%2Caps%2C574&ref=nb_sb_ss_pltr-xclick_1_6"
    HEADERS = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36',
        'Accept-Language': 'en-US,en;q=0.5',
    }

    # MAKING REQUEST
    webpage = requests.get(URL, headers=HEADERS).text
    soup = BeautifulSoup(webpage, 'lxml')
    
    # LINKS (ANCHOR TAGS)
    links = soup.find_all('a', attrs={'class':'a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal'})
    # NAVIGATING LINKS TO GET HREF LINKS
    links_list = []
    for link in links:
        links_list.append(link.get('href'))

    # LOOPING LINKS_LIST TO GET DETAILS
    dictionary_info = {'titles': [], 'prices': [], 'ratings': [], 'reviews': [], 'availability': [],}
    for link in links_list:
        new_webpage = requests.get('https://www.amazon.com'+ link, headers=HEADERS).content # OR .content
        new_soup = BeautifulSoup(new_webpage, 'lxml')
        # Filling Information
        dictionary_info['titles'].append(get_title(new_soup))
        dictionary_info['prices'].append(get_price(new_soup))
        dictionary_info['ratings'].append(get_rating(new_soup))
        dictionary_info['reviews'].append(get_review_count(new_soup))
        dictionary_info['availability'].append(get_availability(new_soup))
    amazon_df = pd.DataFrame.from_dict(dictionary_info)
    amazon_df['titles'].replace('', np.nan, inplace=True)
    amazon_df = amazon_df.dropna(subset=['titles'])
    amazon_df.to_csv('amazon_data.csv', header=True, index=False)

        
    

C:\Users\AllahRakha\AppData\Local\Temp\ipykernel_11472\4128810479.py:33: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  amazon_df['titles'].replace('', np.nan, inplace=True)
